In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('Test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 11:33:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
ordersDF = spark.read.json('retail_db_json-main/orders/part-r-00000-990f5773-9005-49ba-b670-631286032674')

In [3]:
ordersDF.printSchema()

root
 |-- order_customer_id: long (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- order_status: string (nullable = true)



In [4]:
ordersDF.show(5,truncate = False)

+-----------------+---------------------+--------+---------------+
|order_customer_id|order_date           |order_id|order_status   |
+-----------------+---------------------+--------+---------------+
|11599            |2013-07-25 00:00:00.0|1       |CLOSED         |
|256              |2013-07-25 00:00:00.0|2       |PENDING_PAYMENT|
|12111            |2013-07-25 00:00:00.0|3       |COMPLETE       |
|8827             |2013-07-25 00:00:00.0|4       |CLOSED         |
|11318            |2013-07-25 00:00:00.0|5       |COMPLETE       |
+-----------------+---------------------+--------+---------------+
only showing top 5 rows



In [5]:
ordersDF.count()

68883

In [6]:
ordersDF. \
    groupby('order_date'). \
    agg(count(lit(1)).alias('order_count')).show(truncate = False)

+---------------------+-----------+
|order_date           |order_count|
+---------------------+-----------+
|2013-08-13 00:00:00.0|73         |
|2013-10-12 00:00:00.0|162        |
|2013-11-15 00:00:00.0|135        |
|2014-03-19 00:00:00.0|130        |
|2013-09-16 00:00:00.0|121        |
|2013-09-20 00:00:00.0|139        |
|2013-12-31 00:00:00.0|266        |
|2013-09-06 00:00:00.0|276        |
|2013-12-24 00:00:00.0|170        |
|2014-01-07 00:00:00.0|163        |
|2013-10-14 00:00:00.0|139        |
|2013-11-11 00:00:00.0|246        |
|2014-01-27 00:00:00.0|163        |
|2014-01-29 00:00:00.0|158        |
|2014-02-14 00:00:00.0|174        |
|2013-09-07 00:00:00.0|215        |
|2013-09-11 00:00:00.0|109        |
|2013-10-20 00:00:00.0|154        |
|2014-01-09 00:00:00.0|207        |
|2014-03-03 00:00:00.0|266        |
+---------------------+-----------+
only showing top 20 rows



In [7]:
ordersDF. \
    groupby('order_date'). \
    agg(count(lit(1)).alias('orders_count')).count()

364

In [8]:
ordersDF. \
    rollup('order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_date'). \
    show()

+--------------------+-----------+
|          order_date|order_count|
+--------------------+-----------+
|                null|      68883|
|2013-07-25 00:00:...|        143|
|2013-07-26 00:00:...|        269|
|2013-07-27 00:00:...|        202|
|2013-07-28 00:00:...|        187|
|2013-07-29 00:00:...|        253|
|2013-07-30 00:00:...|        227|
|2013-07-31 00:00:...|        252|
|2013-08-01 00:00:...|        246|
|2013-08-02 00:00:...|        224|
|2013-08-03 00:00:...|        183|
|2013-08-04 00:00:...|        187|
|2013-08-05 00:00:...|        153|
|2013-08-06 00:00:...|        258|
|2013-08-07 00:00:...|        203|
|2013-08-08 00:00:...|        154|
|2013-08-09 00:00:...|        125|
|2013-08-10 00:00:...|        270|
|2013-08-11 00:00:...|        154|
|2013-08-12 00:00:...|        255|
+--------------------+-----------+
only showing top 20 rows



In [9]:
ordersDF. \
    rollup('order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_date'). \
    count()

365

In [10]:
ordersDF. \
    groupBy(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    show()

+-----------+--------------------+-----------+
|order_month|          order_date|order_count|
+-----------+--------------------+-----------+
|     201307|2013-07-25 00:00:...|        143|
|     201307|2013-07-26 00:00:...|        269|
|     201307|2013-07-27 00:00:...|        202|
|     201307|2013-07-28 00:00:...|        187|
|     201307|2013-07-29 00:00:...|        253|
|     201307|2013-07-30 00:00:...|        227|
|     201307|2013-07-31 00:00:...|        252|
|     201308|2013-08-01 00:00:...|        246|
|     201308|2013-08-02 00:00:...|        224|
|     201308|2013-08-03 00:00:...|        183|
|     201308|2013-08-04 00:00:...|        187|
|     201308|2013-08-05 00:00:...|        153|
|     201308|2013-08-06 00:00:...|        258|
|     201308|2013-08-07 00:00:...|        203|
|     201308|2013-08-08 00:00:...|        154|
|     201308|2013-08-09 00:00:...|        125|
|     201308|2013-08-10 00:00:...|        270|
|     201308|2013-08-11 00:00:...|        154|
|     201308|

In [11]:
ordersDF. \
    groupBy(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    count()

364

In [12]:
ordersDF. \
    rollup(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    show()

+-----------+--------------------+-----------+
|order_month|          order_date|order_count|
+-----------+--------------------+-----------+
|       null|                null|      68883|
|     201307|                null|       1533|
|     201307|2013-07-25 00:00:...|        143|
|     201307|2013-07-26 00:00:...|        269|
|     201307|2013-07-27 00:00:...|        202|
|     201307|2013-07-28 00:00:...|        187|
|     201307|2013-07-29 00:00:...|        253|
|     201307|2013-07-30 00:00:...|        227|
|     201307|2013-07-31 00:00:...|        252|
|     201308|                null|       5680|
|     201308|2013-08-01 00:00:...|        246|
|     201308|2013-08-02 00:00:...|        224|
|     201308|2013-08-03 00:00:...|        183|
|     201308|2013-08-04 00:00:...|        187|
|     201308|2013-08-05 00:00:...|        153|
|     201308|2013-08-06 00:00:...|        258|
|     201308|2013-08-07 00:00:...|        203|
|     201308|2013-08-08 00:00:...|        154|
|     201308|

In [13]:
ordersDF. \
    rollup(date_format('order_date', 'yyyyMM').alias('order_month'), 'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    orderBy('order_month', 'order_date'). \
    count()

378

In [14]:
ordersDF. \
    groupby(year(col('order_date')).alias('Year'),date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    show()

+----+------+--------------------+-----------+
|Year| Month|          order_date|order_count|
+----+------+--------------------+-----------+
|2013|201309|2013-09-13 00:00:...|        103|
|2013|201312|2013-12-06 00:00:...|        256|
|2013|201310|2013-10-13 00:00:...|        277|
|2013|201309|2013-09-01 00:00:...|        119|
|2013|201310|2013-10-06 00:00:...|        249|
|2014|201401|2014-01-14 00:00:...|        209|
|2014|201401|2014-01-16 00:00:...|        194|
|2014|201403|2014-03-18 00:00:...|        252|
|2013|201309|2013-09-23 00:00:...|        177|
|2013|201312|2013-12-23 00:00:...|        154|
|2014|201402|2014-02-09 00:00:...|        240|
|2013|201308|2013-08-13 00:00:...|         73|
|2014|201403|2014-03-14 00:00:...|        201|
|2014|201401|2014-01-04 00:00:...|        129|
|2014|201401|2014-01-31 00:00:...|        223|
|2013|201309|2013-09-11 00:00:...|        109|
|2014|201401|2014-01-18 00:00:...|        139|
|2013|201309|2013-09-16 00:00:...|        121|
|2014|201401|

In [15]:
ordersDF. \
    groupby(year(col('order_date')).alias('Year'),date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')). \
    count()

364

In [16]:
ordersDF. \
    rollup(year('order_date').alias('Year'),date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).show()

+----+------+--------------------+-----------+
|Year| Month|          order_date|order_count|
+----+------+--------------------+-----------+
|2013|201312|2013-12-13 00:00:...|        135|
|2013|201311|2013-11-06 00:00:...|        234|
|2013|201309|2013-09-05 00:00:...|        261|
|2013|201310|2013-10-03 00:00:...|        154|
|2013|201311|2013-11-23 00:00:...|        251|
|2014|201402|2014-02-15 00:00:...|        221|
|2013|201308|                null|       5680|
|2013|201308|2013-08-12 00:00:...|        255|
|2013|201309|2013-09-24 00:00:...|        254|
|2013|201309|2013-09-19 00:00:...|        206|
|null|  null|                null|      68883|
|2013|201307|2013-07-31 00:00:...|        252|
|2013|201312|2013-12-31 00:00:...|        266|
|2014|201402|2014-02-20 00:00:...|        145|
|2014|201402|2014-02-07 00:00:...|        228|
|2013|  null|                null|      30662|
|2014|201401|2014-01-19 00:00:...|        217|
|2014|201403|2014-03-14 00:00:...|        201|
|2013|201309|

In [17]:
ordersDF. \
    rollup(year('order_date').alias('Year'),date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).count()

380

In [19]:
ordersDF.show()

+-----------------+--------------------+--------+---------------+
|order_customer_id|          order_date|order_id|   order_status|
+-----------------+--------------------+--------+---------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|
|            12111|2013-07-25 00:00:...|       3|       COMPLETE|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|
|            11318|2013-07-25 00:00:...|       5|       COMPLETE|
|             7130|2013-07-25 00:00:...|       6|       COMPLETE|
|             4530|2013-07-25 00:00:...|       7|       COMPLETE|
|             2911|2013-07-25 00:00:...|       8|     PROCESSING|
|             5657|2013-07-25 00:00:...|       9|PENDING_PAYMENT|
|             5648|2013-07-25 00:00:...|      10|PENDING_PAYMENT|
|              918|2013-07-25 00:00:...|      11| PAYMENT_REVIEW|
|             1837|2013-07-25 00:00:...|      12|         CLOSED|
|         

In [28]:
ordersDF. \
    cube('order_date'). \
    agg(count(lit(1)).alias('order_count')).orderBy('order_date').show()

+--------------------+-----------+
|          order_date|order_count|
+--------------------+-----------+
|                null|      68883|
|2013-07-25 00:00:...|        143|
|2013-07-26 00:00:...|        269|
|2013-07-27 00:00:...|        202|
|2013-07-28 00:00:...|        187|
|2013-07-29 00:00:...|        253|
|2013-07-30 00:00:...|        227|
|2013-07-31 00:00:...|        252|
|2013-08-01 00:00:...|        246|
|2013-08-02 00:00:...|        224|
|2013-08-03 00:00:...|        183|
|2013-08-04 00:00:...|        187|
|2013-08-05 00:00:...|        153|
|2013-08-06 00:00:...|        258|
|2013-08-07 00:00:...|        203|
|2013-08-08 00:00:...|        154|
|2013-08-09 00:00:...|        125|
|2013-08-10 00:00:...|        270|
|2013-08-11 00:00:...|        154|
|2013-08-12 00:00:...|        255|
+--------------------+-----------+
only showing top 20 rows



In [22]:
ordersDF. \
    cube('order_date'). \
    agg(count(lit(1)).alias('order_count')).count()

365

In [30]:
ordersDF. \
    groupby(date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).show()

+------+--------------------+-----------+
| Month|          order_date|order_count|
+------+--------------------+-----------+
|201308|2013-08-18 00:00:...|        199|
|201402|2014-02-02 00:00:...|        192|
|201308|2013-08-19 00:00:...|         93|
|201311|2013-11-23 00:00:...|        251|
|201401|2014-01-20 00:00:...|        203|
|201401|2014-01-27 00:00:...|        163|
|201402|2014-02-18 00:00:...|        219|
|201308|2013-08-16 00:00:...|        131|
|201311|2013-11-08 00:00:...|        170|
|201311|2013-11-19 00:00:...|        188|
|201311|2013-11-25 00:00:...|        133|
|201308|2013-08-04 00:00:...|        187|
|201310|2013-10-18 00:00:...|        129|
|201311|2013-11-06 00:00:...|        234|
|201311|2013-11-13 00:00:...|        236|
|201309|2013-09-11 00:00:...|        109|
|201402|2014-02-20 00:00:...|        145|
|201402|2014-02-08 00:00:...|        163|
|201402|2014-02-19 00:00:...|        268|
|201310|2013-10-16 00:00:...|        127|
+------+--------------------+-----

In [31]:
ordersDF. \
    groupby(date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).count()

364

In [32]:
ordersDF. \
    cube(date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).show()

+------+--------------------+-----------+
| Month|          order_date|order_count|
+------+--------------------+-----------+
|201309|2013-09-05 00:00:...|        261|
|201401|2014-01-01 00:00:...|        135|
|  null|2013-09-23 00:00:...|        177|
|201309|2013-09-22 00:00:...|        192|
|201312|2013-12-23 00:00:...|        154|
|  null|2013-10-05 00:00:...|        215|
|  null|2013-12-27 00:00:...|        162|
|201308|2013-08-09 00:00:...|        125|
|201401|2014-01-16 00:00:...|        194|
|  null|2013-11-15 00:00:...|        135|
|201311|2013-11-15 00:00:...|        135|
|201402|2014-02-01 00:00:...|        278|
|  null|2013-12-17 00:00:...|        132|
|201401|2014-01-10 00:00:...|        241|
|201403|2014-03-11 00:00:...|        204|
|  null|2013-12-20 00:00:...|        259|
|  null|2014-01-04 00:00:...|        129|
|  null|2014-02-13 00:00:...|        203|
|201308|2013-08-03 00:00:...|        183|
|  null|2014-01-15 00:00:...|        243|
+------+--------------------+-----

In [34]:
ordersDF. \
    cube(date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).count()

742

In [35]:
ordersDF. \
    groupby(year('order_date').alias('Year'), date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).show()

+----+------+--------------------+-----------+
|Year| Month|          order_date|order_count|
+----+------+--------------------+-----------+
|2013|201309|2013-09-13 00:00:...|        103|
|2013|201312|2013-12-06 00:00:...|        256|
|2013|201310|2013-10-13 00:00:...|        277|
|2013|201309|2013-09-01 00:00:...|        119|
|2013|201310|2013-10-06 00:00:...|        249|
|2014|201401|2014-01-14 00:00:...|        209|
|2014|201401|2014-01-16 00:00:...|        194|
|2014|201403|2014-03-18 00:00:...|        252|
|2013|201309|2013-09-23 00:00:...|        177|
|2013|201312|2013-12-23 00:00:...|        154|
|2014|201402|2014-02-09 00:00:...|        240|
|2013|201308|2013-08-13 00:00:...|         73|
|2014|201403|2014-03-14 00:00:...|        201|
|2014|201401|2014-01-04 00:00:...|        129|
|2014|201401|2014-01-31 00:00:...|        223|
|2013|201309|2013-09-11 00:00:...|        109|
|2014|201401|2014-01-18 00:00:...|        139|
|2013|201309|2013-09-16 00:00:...|        121|
|2014|201401|

In [36]:
ordersDF. \
    groupby(year('order_date').alias('Year'), date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).count()

364

In [37]:
ordersDF. \
    rollup(year('order_date').alias('Year'), date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).show()

+----+------+--------------------+-----------+
|Year| Month|          order_date|order_count|
+----+------+--------------------+-----------+
|2013|201312|2013-12-13 00:00:...|        135|
|2013|201311|2013-11-06 00:00:...|        234|
|2013|201309|2013-09-05 00:00:...|        261|
|2013|201310|2013-10-03 00:00:...|        154|
|2013|201311|2013-11-23 00:00:...|        251|
|2014|201402|2014-02-15 00:00:...|        221|
|2013|201308|                null|       5680|
|2013|201308|2013-08-12 00:00:...|        255|
|2013|201309|2013-09-24 00:00:...|        254|
|2013|201309|2013-09-19 00:00:...|        206|
|null|  null|                null|      68883|
|2013|201307|2013-07-31 00:00:...|        252|
|2013|201312|2013-12-31 00:00:...|        266|
|2014|201402|2014-02-20 00:00:...|        145|
|2014|201402|2014-02-07 00:00:...|        228|
|2013|  null|                null|      30662|
|2014|201401|2014-01-19 00:00:...|        217|
|2014|201403|2014-03-14 00:00:...|        201|
|2013|201309|

In [38]:
ordersDF. \
    rollup(year('order_date').alias('Year'), date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).count()

380

In [39]:
ordersDF. \
    cube(year('order_date').alias('Year'), date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).show()

+----+------+--------------------+-----------+
|Year| Month|          order_date|order_count|
+----+------+--------------------+-----------+
|2013|201312|2013-12-13 00:00:...|        135|
|null|201308|2013-08-27 00:00:...|        188|
|null|  null|2013-11-04 00:00:...|        254|
|null|  null|2013-11-22 00:00:...|        249|
|null|  null|2013-12-07 00:00:...|        234|
|null|201401|2014-01-17 00:00:...|        149|
|null|  null|2014-03-08 00:00:...|        100|
|null|  null|2014-03-20 00:00:...|        177|
|2013|201311|2013-11-06 00:00:...|        234|
|null|  null|2013-07-27 00:00:...|        202|
|2013|  null|2013-09-22 00:00:...|        192|
|2013|  null|2013-12-08 00:00:...|        141|
|2013|  null|2013-12-25 00:00:...|        217|
|null|  null|2014-03-19 00:00:...|        130|
|2013|201309|2013-09-05 00:00:...|        261|
|2013|201310|2013-10-03 00:00:...|        154|
|2013|  null|2013-08-17 00:00:...|        253|
|2013|  null|2013-10-22 00:00:...|        125|
|null|  null|

In [40]:
ordersDF. \
    cube(year('order_date').alias('Year'), date_format('order_date','yyyyMM').alias('Month'),'order_date'). \
    agg(count(lit(1)).alias('order_count')).count()

1485